# MOM6/CESM Ocean Model Analysis

This notebook shows how to load and analyze ocean data from an out-of-the-box MOM6/CESM G-case simulation (coupled ocean ocean/sea ice). 

**NOTE**: MOM6/CESM is not ready to be used for research. 

In [ ]:
%matplotlib inline

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import holoviews as hv
import datashader
from holoviews.operation.datashader import regrid, shade, datashade

hv.extension('bokeh', width=100)

## Create and Connect to Dask Distributed Cluster

This will launch a cluster of virtual machines in the cloud.

In [ ]:
from dask.distributed import Client, progress
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=10)
cluster

👆 Don't forget to click this link to get the cluster dashboard

In [ ]:
client = Client(cluster)
client

## Load MOM6/CESM Data

This data is stored in [xarray-zarr](http://xarray.pydata.org/en/latest/io.html#zarr) format in Google Cloud Storage.
This format is optimized for parallel distributed reads from within the cloud environment.

It may take up to a minute to initialize the dataset when you run this cell.

In [ ]:
# Load with Cloud object storage
import gcsfs
gcsmap = gcsfs.mapping.GCSMap('pangeo-data/MOM6.cesm/')
ds = xr.open_zarr(gcsmap, decode_times=False)

# Print dataset
print(ds)

## Visualize SST Data with Holoviews and Datashader

In [ ]:
%opts QuadMesh [width=1000 height=800]
qm = hv.QuadMesh((ds.geolon[-1].values, ds.geolat[-1].values, ds.SST[-1].values))
datashade(qm, precompute=True, cmap=plt.cm.jet)

## Visualize SSS Data with Holoviews and Datashader

In [ ]:
qm = hv.QuadMesh((ds.geolon[-1].values, ds.geolat[-1].values, ds.SSS[-1].values))
datashade(qm, precompute=True, cmap=plt.cm.Spectral_r)

## Data reduction

Here we make a data reduction by taking the time of SST and SSS. This demonstrates how the cluster distributes the reads from storage.

In [ ]:
SST_mean = ds.SST.mean(dim=('time'))
SST_mean

In [ ]:
SSS_mean = ds.SSS.mean(dim=('time'))
SSS_mean

In [ ]:
%time SST_mean.load()

In [ ]:
qm = hv.QuadMesh((ds.geolon[-1].values, ds.geolat[-1].values, SST_mean))
datashade(qm, precompute=True, cmap=plt.cm.jet)

In [ ]:
%time SSS_mean.load()

In [ ]:
qm = hv.QuadMesh((ds.geolon[-1].values, ds.geolat[-1].values, SSS_mean))
datashade(qm, precompute=True, cmap=plt.cm.Spectral_r)